
# Projet 2

In [1]:
# import des modules usuels
import numpy as np
import pandas as pd

# options d'affichage
pd.set_option("display.min_rows", 16)

In [2]:
# chargement et traitement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
geo = geo.sort_index()
geo[['Latitude', 'Longitude']] = geo["geo_point_2d"].str.extract("(.*), (.*)").astype(float)
geo.head()
geo.shape

(36742, 12)

**Projet maison n° 1**

- Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.
- Ecrivez une fonction qui détermine la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ajoutez une fonction de conversion pour pouvoir utiliser la première fonction avec un GPS (degrés, minutes, secondes).

In [3]:
# Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.

geo.insert(6, "CP Ville", geo['Code Postal'] + " " + geo['Commune'])
#geo['CP Ville'] = geo['Code Postal'] + " " + geo['Commune']

geo.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,CP Ville,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude
Code INSEE,,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,01400 L'ABERGEMENT-CLEMENCIAT,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,01640 L'ABERGEMENT-DE-VAREY,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,01500 AMBERIEU-EN-BUGEY,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,01330 AMBERIEUX-EN-DOMBES,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,01300 AMBLEON,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320


In [4]:
# Ecrivez une fonction qui détermine la commune la plus proche d'un point à partir de sa latitude et sa longitude

def distance_between_two_coords(lat1, long1, lat2, long2):
    return ((lat1-lat2)**2 + (long1-long2)**2)**(1/2)

# fonction recherche de ville
def ville(lat, long):
    shortest_distance=10000000
    shortest_city=""
    for index, row in geo.iterrows():
        lat1, long1 = row['Latitude'], row['Longitude']
        distance = distance_between_two_coords(lat1, long1, lat, long)
        if distance < shortest_distance:
            shortest_distance = distance
            shortest_city = row['Commune']
    return shortest_city


In [5]:
# Ajoutez une fonction de conversion pour pouvoir utiliser la première fonction avec un GPS (degrés, minutes, secondes).
# conversion degrés, minutes, secondes => décimal

def dms2dec(deg, mn, sec):
    return (deg + mn / 60 + sec / 3600) if deg >=0 else (deg - mn / 60 - sec / 3600)

In [6]:
# on applique la fonction à une coordonnée tirée au hasard
# init du random
np.random.seed(0)

a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = (b - a) * np.random.random() + a
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
long = (b - a) * np.random.random() + a

ville(lat, long)

'BOSJEAN'

In [7]:
# à partir de coordonnées GPS précises
ville(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

'PALAISEAU'

**Projet maison n° 2**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python json pour parser la colonne "geo_shape".
- Donner le décompte des valeurs accédées avec la clé "type".
- Donner le décompte des longueurs des listes accédées avec la clé "coordinates".
- Quelle commune est la plus complexe géométriquement ?
- Quelle commune est la seconde la plus complexe géométriquement ?
- Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : installer la librairie shapely, utiliser la classe Polygon de shapely.geometry. Sur Windows shapely peut nécessiter d'installer la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [8]:
# Utiliser la librairie Python json pour parser la colonne "geo_shape"
import json

# Fonctions renvoyant les valeurs des clés "type" et "coordinates"
def json_load(content, field):
    dict_content = json.loads(content)
    return dict_content[field]


In [9]:
# Donner le décompte des valeurs accédées avec la clé "type"
geo["type"] = geo['geo_shape'].apply(json_load, field='type')

# Décompte des valeurs de clé "type"
geo["type"].value_counts()


Polygon         36670
MultiPolygon       72
Name: type, dtype: int64

In [10]:
# Donner le décompte des longueurs des listes accédées avec la clé "coordinates"

geo["coordinates"] = geo['geo_shape'].apply(json_load, field='coordinates')
geo["count_coordinates"] = geo["coordinates"].apply(len)

# Longueur des listes avec la clé "coordinates"
geo["count_coordinates"].value_counts()

1    36660
2       80
3        1
4        1
Name: count_coordinates, dtype: int64

In [11]:
geo.head(5)

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,CP Ville,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,type,coordinates,count_coordinates
Code INSEE,,,,,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,01400 L'ABERGEMENT-CLEMENCIAT,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114,Polygon,"[[[4.92627342692396, 46.1200517234555], [4.921...",1
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,01640 L'ABERGEMENT-DE-VAREY,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017,Polygon,"[[[5.430089291292173, 45.98275997279625], [5.4...",1
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,01500 AMBERIEU-EN-BUGEY,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926,Polygon,"[[[5.386190824025674, 45.930927620632], [5.374...",1
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,01330 AMBERIEUX-EN-DOMBES,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273,Polygon,"[[[4.895580369408855, 45.97991618124606], [4.8...",1
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,01300 AMBLEON,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320,Polygon,"[[[5.614854625053272, 45.734638855121595], [5....",1


In [12]:
# Quelle commune est la plus complexe géométriquement ?

most_complex = geo.Commune.loc[geo['count_coordinates'] == 4]
print(f' La commune la plus complexe géométriquement : {most_complex.values[0]}')

 La commune la plus complexe géométriquement : HYERES


In [13]:
# Quelle commune est la seconde la plus complexe géométriquement ?

second_most_complex = geo.Commune.loc[geo['count_coordinates'] == 3]
print(f' La seconde commune la plus complexe géométriquement : {second_most_complex.values[0]}')

 La seconde commune la plus complexe géométriquement : VAL-DE-REUIL


In [14]:
# Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?

poly_two_coord = geo.Commune.loc[(geo['type'] == "Polygon") & (geo['count_coordinates'] == 2)]
print('Les villes de type Polygon mais dont la longueur des listes est de 2 : ')
for cities in poly_two_coord.values:
    print(cities)

Les villes de type Polygon mais dont la longueur des listes est de 2 : 
COYOLLES
EYGLIERS
LA BASTIDE-DE-SEROU
PLOUBALAY
SARTENE
CALENZANA
LANGOGNE
SUNDHOUSE
EGUISHEIM
BEAUVERNOIS


In [15]:
# Pour ces villes vérifier que le premier polygone contient bien le second (enclave)

from shapely.geometry import Polygon

cities = geo.Commune.loc[(geo['type'] == "Polygon") & (geo['count_coordinates'] == 2)].tolist()
for city in cities:
    coordinates = geo.coordinates.loc[geo['Commune'] == city].values[0]
    first_polygon = Polygon(coordinates[0])
    second_polygon = Polygon(coordinates[1])
    print('Le premier polygone contient le second pour '+ city + ' : '+str(first_polygon.contains(second_polygon)))

Le premier polygone contient le second pour COYOLLES : True
Le premier polygone contient le second pour EYGLIERS : True
Le premier polygone contient le second pour LA BASTIDE-DE-SEROU : True
Le premier polygone contient le second pour PLOUBALAY : True
Le premier polygone contient le second pour SARTENE : True
Le premier polygone contient le second pour CALENZANA : True
Le premier polygone contient le second pour LANGOGNE : True
Le premier polygone contient le second pour SUNDHOUSE : True
Le premier polygone contient le second pour EGUISHEIM : True
Le premier polygone contient le second pour BEAUVERNOIS : True
